In [1]:
from pathlib import Path
from scipy.io import loadmat
from datetime import date
import os

keyword = 'imdb'

## MATLAB's datenum is different from Python's date by a term 366
date_difference_matlab = -366
x = loadmat('{0}/{0}.mat'.format(keyword))

In [2]:
val = x[keyword][0,0]
dobs = val['dob']
photos_taken = val['photo_taken']
names = val['name']
full_paths = val['full_path']
genders = val['gender']

In [3]:
source = '/home/deeplearn/Desktop/{}_crop'.format(keyword)
destination = 'IMDb_Dataset'
create_symbolic_links = True
verbose = False

In [4]:

get_date = lambda x : date.fromordinal(x+date_difference_matlab)

def get_destination(destination, gender, name):
    return os.path.join(destination, 'male' if gender == 1.0 else 'female', name)

n_faces = len(dobs[0])

for s in ['male', 'female']:
    if not os.path.exists(os.path.join(destination, s)):
        os.makedirs(os.path.join(destination, s))
count = 0
for i in range(n_faces):
    
    # Date klarar bara av datum efter ett visst datum
    try:
        date_of_birth = get_date(dobs[0, i])
    except:
        continue
    date_of_photo = date(photos_taken[0, i], 7, 1)
    age = (date_of_photo-date_of_birth).days//365
    path = Path(full_paths[0,i][0])
    gender = genders[0,i]
    if gender not in (0.0, 1.0):
        count += 1
        continue
    if verbose:
        print(age,
              path,
              names[0,i][0],
              gender)
    if create_symbolic_links:
        _, name = os.path.split(path)
        dest_path = Path(get_destination(destination, gender, name))
        os.symlink(source / path, dest_path)
print(count)

8462
